## 초기 system prompt 설정 및 벡터 store 생성

In [1]:
!pip install langchain_community

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

permission = "frontend"

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embedding_function=embedding_model)

C:\Users\miny0\AppData\Local\Temp\ipykernel_23472\2660580404.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\miny0\AppData\Local\Temp\ipykernel_23472\2660580404.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embed

## 초기 질문 리스트 가져오기

In [3]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal(f"./company_questions_{permission}.jsonl")
initial_questions

['모델 성능 평가 보고서에서 Random Forest 모델의 정확도는 어떻게 되나요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준 섹션에서 F1 점수의 기준은 무엇인가요?',
 '모델 성능 평가 보고서의 3. 실험/테스트 절차에서 훈련 데이터와 검증 데이터의 비율은 어떻게 되나요?',
 '모델 성능 평가 보고서에서 제안된 개선 방안 중 데이터 불균형 해소를 위한 방법은 무엇인가요?',
 '모델 성능 평가 보고서의 5. 결과 요약 및 검증 포인트에서 Random Forest 모델의 ROC-AUC 값은 얼마인가요?',
 '모델 성능 평가 보고서의 6. 리스크/한계 및 개선 제안 섹션에서 언급된 과적합의 리스크는 무엇인가요?',
 '모델 성능 평가 보고서의 2. 모델/알고리즘 범위에서 사용된 알고리즘은 어떤 것들이 있나요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준에서 혼동 행렬의 구성 요소는 무엇인가요?',
 '모델 성능 평가 보고서의 5. 결과 요약 및 검증 포인트에서 검증 포인트로 제시된 항목은 무엇인가요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준에서 Accuracy의 기준은 얼마인가요?',
 '모델 성능 평가 보고서의 3. 실험/테스트 절차에서 하이퍼파라미터 튜닝이 완료되었는지 확인하는 체크리스트 항목은 무엇인가요?',
 '모델 성능 평가 보고서의 6. 리스크/한계 및 개선 제안에서 데이터 편향의 리스크는 어떤 상황에서 발생할 수 있나요?',
 '데이터 수집 단계에서 데이터 소스 식별 및 목록화는 어떻게 진행되나요?',
 '데이터 전처리에서 결측치 처리 방법은 어떤 기준으로 결정하나요?',
 '데이터 저장 단계에서 선택할 수 있는 저장소의 종류는 무엇인가요?',
 '데이터 분석 단계에서 정의해야 하는 KPI의 예시는 무엇인가요?',
 '데이터 배포 단계에서 데이터 접근 권한 관리는 어떻게 이루어지나요?',
 '데이터 품질 기준에서 수집된 데이터의 정확성 및 완전성을 확인하는 방법은 무엇인가요?',
 '보안/프라이버

### 기본 문서 검색 질문 생성 체인

In [25]:
import random

complex2 = ["구체적으로(ex. '기술적 기여도 평가에서 A/B 테스트 도입이 긍정적인 결과를 가져온 사례에 대해 자세히 설명해 주실 수 있나요?', '문제 해결 능력 강화를 위한 후속 조치 제안 중 '문제 해결 세션'의 구체적인 진행 방식은 어떻게 돼?','프론트엔드팀의 추가 인력 투입이나 교육 실행 시 예상되는 효과나 리스크는 무엇이 있을까?')", "아주 짧고 간결하게 5~10글자 내로(ex. '프론트엔드 팀 업무 알려줘', '업무 인계 문서 찾아줘', '프론트엔드 팀 기술 스택 알려줘봐')"]

def pick_complex2() -> str:
    return random.choices(complex2)[0]

pick_complex2()

"아주 짧고 간결하게 5~10글자 내로(ex. '데이터AI 팀 업무 알려줘', '업무 인계 문서 찾아줘', '데이터AI 팀 기술 스택 알려줘봐')"

In [26]:
def question_chain_setting():
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
            다음 제공되는 대화 내역을 기반으로 다음에 나올 수 있는 사용자가 궁금해할 만한 후속 질문 하나를 사용자 입장에서 질문하세요.
            이전에 나왔던 질문은 최대한 만들지 말고, 연결된 질문으로 조금이라도 다른 질문을 만들어주세요.
            질문은 반드시 {complex} 만들어야 합니다.

            대화 내역: {history}

            절대로 AI입장에서의 질문을 만들지 마세요.
            ex) 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.
            ex) 추가적으로 궁금한 사항이 있으신가요?
            ex) 데이터 접근 권한 관리 방침에 대해 더 궁금한 사항이 있으면 말씀해 주세요.
            ex) 안녕하세요. 다른 궁금한 점이 있으신가요?
            ex) 안녕하십니까? 도움이 필요하신 부분이 있으신가요?
            ex) ~~ 예시는 다음과 같습니다.


            답변형태는 다음과 같이 해주세요.
            ex) 빌드 결과물의 확인 방법이나 배포 과정에 대해 알고 싶어.
            ex) 문서화 시 유의사항 알려줄 수 있나요?
            ex) 빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 뭐야?
            ex) 모델 개발 & 성능 보고서에서 사용된 프로그래밍 언어가 뭐야?
        """
    )

    question_chain = basic_prompt | llm | StrOutputParser()

    return question_chain

## 일상 질문 의도 다양화 로직

In [46]:
import random

INTENTS = [
    "인사/감상", "영상/음악", "운동/스트레칭", "생산성/타이머",
    "생활 팁/정리", "UX(배경/위젯)", "퀴즈/잡학", "날씨/우산 판단",
    "음식"  # <-- 과다 시 일단 제거하거나 가중치↓
]

def pick_intent() -> str:
    # 최근 과다 의도에 역가중치 (음식이 많다면 자동으로 다른 의도 선택)
    return random.choices(INTENTS)[0]

pick_intent()

'날씨/우산 판단'

### 일상 질문 복잡성 다양화 로직

In [52]:
complex = ["구체적으로(ex. '오늘 간단하게 할 수 있는 취미가 뭐가 있을까?', '여름에 어울리는 시원한 음료 추천해줘.','마음이 편안해지는 방법')", "아주 짧고 간결하게 5~10글자 내로(ex. '안녕', '오늘 뭐하지', '오늘 운동 추천좀')"]

def pick_complex() -> str:
    return random.choices(complex)[0]

pick_complex()

"구체적으로(ex. '오늘 간단하게 할 수 있는 취미가 뭐가 있을까?', '여름에 어울리는 시원한 음료 추천해줘.','마음이 편안해지는 방법')"

### 일상 질문 생성 체인

In [53]:
def simple_chain_setting():
    # 표준 말투 유지 + 중복 표현 회피 보조
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.3,
        presence_penalty=0.6,   # 이미 썼던 표현 회피 유도
        frequency_penalty=0.3,  # 반복 억제
    )

    basic_prompt = PromptTemplate.from_template(
        """
다음은 '사내 직원이 챗봇(무생물)에게' 던질 **캐주얼 인사/잡담/가벼운 추천형** 1문장을 생성하는 작업입니다.

[최우선 지시(원래 규칙 포함·강화)]
- 문서/보고서/회사자료/회사정보/설정/보안/권한/기능 설명/도움말/사용법 관련 질문·요청 금지
- 문서에 대해 물어보는 문장 절대 금지 (예: "문서 보여줘", "보고서 찾아줘", "사내 자료 요약해줘" 등)
- 사람(동료/개인)에게 묻는 질문 금지
  - 개인 경험/감정/계획 확인: "주말에 뭐 할 계획이신가요?", "오늘 뭐했어?", "기분 어때?", "어제 어땠어?"
  - 2인칭 친소/경어형 질문: "~했어?", "~하셨나요?", "~계신가요?"
  - 상호작용 요구(사람 기준): "같이 ~할래요?", "우리 ~할까요?"
- 의도는 반드시 다음에 맞추세요: {intent}
- 질문은 {complex} 만들어야 합니다.

[허용 의도: 컴퓨터에게 할 법한 표현만]
- 인사·가벼운 감상: "안녕!", "오늘 날씨가 꿀꿀하네..", "날씨 좋네!"
- 가벼운 추천/요청: "오늘 저녁 뭐 먹을까?", "퇴근 후 뭐 할까?", "주말에 뭐 하면 좋을까?"

[중복/유사도 회피]
- 아래 {history}에 포함된 과거 문장/질문과 **동일하거나 매우 유사한 의미/어휘/어순**을 피하세요.
- 특히 다음을 지키세요:
  1) 과거 문장과 5-그램 연속 일치가 2회 이상 발생하지 않도록 표현을 바꾸세요.
  2) 핵심 어휘의 최소 40% 이상을 **동의어/대체 표현**으로 교체하세요.
  3) 문장 종결(어미)와 지시 동사를 바꿔 **표현 다양성**을 확보하세요. (예: "~추천해줘" 반복 금지)

[출력 형식]
- 한국어, **오직 1문장**만 출력 (설명/따옴표/불릿/접두어 금지)
- 사람 대상 과도한 공손말 금지("~계신가요?" 등)
- 가능하면 컴퓨터 지시형 동사 사용: "~알려줘", "~정해줘", "~골라줘", "~하나 뽑아줘", "~판단해줘", "~틀어줘", "~켜줘" 등
- **예시 문장들의 어휘/어순을 그대로 복사하지 마세요.**

[대화내용(중복 회피 기준)]
{history}

[참고 예시(그대로 쓰지 말 것)]
- 오늘 점심 뭐 먹을지 하나만 추천해줘.
- 주말에 가볍게 할 만한 취미 하나 알려줘.
- 오늘은 좀 피곤하네, 기분 전환용 음악 장르 하나 골라줘.
- 퇴근 후 간단히 볼만한 영화 한 편만 추천해줘.
- 요즘 날씨 오락가락하네, 우산 챙길지 판단 좀 도와줘.

[원치 않는 출력(절대 금지)]
- 요즘 날씨가 정말 좋네요, 주말에 뭘 할 계획이신가요?
- 오늘 뭐했어?
- 어제는 어디 다녀오셨어요?
- 주말에 같이 등산 갈래요?

위 모든 규칙을 지켜 **1문장만** 출력하세요.
        """
    )

    simple_chain = basic_prompt | llm | StrOutputParser()
    return simple_chain


### 검색 쿼리 정제 체인

In [54]:
def query_setting_for_internal_documents():
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    query_prompt = PromptTemplate.from_template(
        """
       유저의 채팅 히스토리가 주어집니다.

       **중요**: 이전 대화 맥락을 반드시 고려해서, 사내 문서 검색에 적합한 쿼리를 생성하세요.
       - 현재 대화가 이전 대화와 연관되어 있다면, 이전 맥락을 반영하여 문서 검색을 위한 구체적인 쿼리를 만들어주세요.

       **복잡한 질문을 어떻게 정제하고 분리할지에 대한 예시**:
       1. 사용자가 "API 서버의 설정과 관련된 문서와, 데이터베이스 연결 방법에 대해 알고 싶어."라고 물었을 경우:
          - **정제된 쿼리**: "API 서버 설정 방법"과 "데이터베이스 연결 방법"
          - 이처럼 복잡한 질문을 각각 분리하여 **개별적인 검색 쿼리**를 생성해야 합니다.

       2. 사용자가 "사내 보상 정책과 휴가 정책을 한 번에 찾아볼 수 있을까?"라고 물었을 경우:
          - **정제된 쿼리**: "사내 보상 정책"과 "사내 휴가 정책"
          - 질문이 하나지만 여러 부분으로 나눠서 **별개의 쿼리**를 생성합니다.

       3. 사용자가 "다음 프로젝트의 개발 환경과 관련된 문서, 그리고 테스팅 방법에 대해 알려줘."라고 물었을 경우:
          - **정제된 쿼리**: "다음 프로젝트의 개발 환경"과 "테스팅 방법"
          - 복잡한 주제일 경우, 각 주제별로 **검색 쿼리**를 명확하게 분리합니다.

        4. 사용자가 이전 대화에서 "사내 휴가 정책이 어떻게 돼?"라고 물어보고 나서 답변을 받고 나서, "좀 더 자세히 알려줘. 그리고 인사 정책도 알려줘."라고 물었을 경우:
          - **정제된 쿼리**: "사내 휴가 정책에 대한 자세한 내용"과 "사내 인사 정책"
          - 이전 대화 내역과 이번 질문의 맥락을 고려하여 **검색 쿼리**를 생성합니다.

       **주의사항**:
       - 이전 대화에서 이미 답변이 나온 질문에 대해서는 다시 질문을 만들지 마세요.
       - 쿼리는 1개 또는 여러 개일 수 있습니다.

       대화 히스토리: {history}

       JSON 반환 형태:
       {{"queries": ["사내 문서 검색 쿼리 1", "사내 문서 검색 쿼리 2", ...]}}
       """
    )

    # 이후 이 부분에 실제로 대화 히스토리를 기반으로 쿼리들을 생성하는 로직이 들어갈 것입니다.
    query_chain = query_prompt | llm | StrOutputParser()

    return query_chain

In [55]:
question_chain = question_chain_setting()
query_chain = query_setting_for_internal_documents()
simple_chain = simple_chain_setting()

### 기존 학습 데이터에 있는 초기 일상 질문 리스트

In [1]:
hello_list = ['안녕!', '반가워!', '안녕하세요!', '오늘 날씨가 꿀꿀하네..', '반가워용', '반갑습니다',
 '안녕', '반가워', '안녕하세요', '반갑습니다', '날씨 좋네!', '요즘 날씨가 정말 변덕스럽네.',
 '오늘 점심 메뉴 추천해줘.', '오늘 저녁 뭐 먹을까?', '오늘 퇴근 후 뭐 할까?', '주말에 뭐 할지 고민 중이야.',
 '오늘은 좀 피곤하네.', '이번 주 주말에 뭐 할까?', '오늘 뭐 할지 모르겠어.', '요즘 뭐 재미있는 거 있어?',
 '주말에 영화 볼까?', '오늘 기분이 좀 이상해.', '이따가 뭐 먹을까?']

hello_list2 = ['오늘 기분이 어떤지 궁금해!',
 '요즘 인기 있는 음악 추천해 줄래?',
 '좋은 영화 있으면 알려줘!',
 '요즘 트렌드인 패션 아이템 뭐가 있을까?',
 '사무실에서 듣기 좋은 팟캐스트 추천해 줄 수 있어',
 '휴식 시간에 할 만한 간단한 게임 있으면 알려줘!',
 '최근에 화제가 된 뉴스는 뭐가 있을까?',
 '오늘 날씨 어때?',
 '내가 가볼 만한 여행지 추천해 줘!',
 '재밌는 책 읽고 싶은데, 추천할 만한 거 있어?',
 '요즘 인기 있는 TV 프로그램 있으면 알려줘!',
 '간단한 요리 레시피 알려줄래?',
 '주말에 놀러 갈 만한 장소 추천해 줘!',
 '요즘 핫한 카페나 맛집 정보 있으면 알려줘!',
 '기분 전환할 만한 좋은 방법이 있을까?',
 '새로운 취미를 시작해 보고 싶은데 뭐가 좋을까?',
 '사무실에 어울리는 음악 플레이리스트 추천해 줘!',
 '감동적인 영화 있으면 한 편 추천해 줘!',
 '인기 있는 앱이나 웹사이트 알려줄래?',
 '오늘의 긍정적인 메시지 하나 던져 줄래?',
 '가벼운 스트레칭 방법 추천해 줘!',
 '재밌는 유튜브 채널 알려줄 수 있어?',
 '감상할 만한 미술 전시회 있으면 추천해 줘!',
 '건강한 스낵 아이디어가 있다면 알려줘!',
 '요즘 사람들 사이에서 인기 있는 활동이 뭐지?',
 '오늘 날씨 어때?',
 '요즘 핫한 영화 추천해줄 수 있어?',
 '좋아하는 음악 있으면 알려줘!',
 '최근에 나온 책 중에 읽어볼 만한 거 있어?',
 '어떤 유튜브 채널 재미있어?',
 '오늘 기분 좋게 해줄 만한 명언 하나 알려줘!',
 '나에게 어울리는 디저트 추천해줘!',
 '주말에 할 만한 활동이 있을까?',
 '요즘 인기 있는 드라마 뭐야?',
 '나랑 잘 맞는 취미가 뭐라고 생각해?',
 '스트레스 해소하는 방법 몇 가지 알려줘!',
 '재미있는 팟캐스트 추천해줄래?',
 '다음에 가볼 만한 여행지 있으면 알려줘!',
 '내가 좋아할 만한 게임 뭐 있을까?',
 '나에게 맞는 카페 스타일은 뭐일까?',
 '요즘 트렌디한 패션 아이템 추천해줘!',
 '분위기 좋은 음악 추천해줄 수 있어?',
 '나한테 어울리는 색상은 뭐라고 생각해?',
 '새로운 레시피 하나 추천해줘!',
 '가끔 찾고 싶은 동기부여 영상 있어?',
 '앞으로 주목할 만한 기술 트렌드 뭐가 있을까?',
 '요즘 유행하는 운동 방법 알려줘!',
 '내가 좋아할 만한 SNS 계정 있으면 추천해줘!',
 '나에게 어울릴 것 같은 차 한 잔 추천해줘!',
 '여름에 듣기 좋은 음악 리스트 만들어줄래?',
 '오늘 기분 좋은 음악 추천해줄 수 있어?',
 '최근에 인기 있는 영화 어떤 게 있을까?',
 '요즘 핫한 책 제목 좀 알려줘!',
 '주말에 하기 좋은 취미 활동은 뭐가 있을까?',
 '새로운 팟캐스트 추천해줄 수 있어?',
 '가볍게 읽기 좋은 웹툰 뭐가 있을까?',
 '요즘 유행하는 요리법 알고 있어?',
 '최근에 나온 게임 중에 추천할 만한 게 있을까?',
 '오늘의 날씨에 딱 맞는 음악이 있을까?',
 '스트레스 풀기 좋은 영화 한 편 추천해줘!',
 '출퇴근길에 듣기 좋은 음악 리스트 좀 만들어줄래?',
 '여름에 가기 좋은 여행지 추천해줄 수 있어?  ',
 '기분 전환에 좋은 활동 뭐가 있을까?  ',
 '맛있는 디저트 레시피 알고 있어?  ',
 '친구한테 선물하기 좋은 아이템 추천해줘!  ',
 '요즘 사람들이 많이 하는 운동 뭐가 있을까?  ',
 '겨울에 추천하는 따뜻한 음료는 뭐가 있을까?  ',
 '새로운 앱 중에 유용한 거 있어?  ',
 '주말에 볼 만한 드라마 추천해줄 수 있어?  ',
 '간단하게 만들 수 있는 스낵 레시피 알고 있어?  ',
 '특별한 날에 어울리는 배경 음악 추천해줘!  ',
 '근사한 카페 정보 좀 알려줄 수 있어?  ',
 '요즘 인싸템으로 뭐가 유행하고 있어?  ',
 '간단한 자기계발 팁 알려줄 수 있어?  ',
 '혼자 여행하기 좋은 장소 추천해줘!',
 '오늘 기분이 어떤지 알려줄 수 있어?  ',
 '요즘 인기 있는 영화 추천해 줄래?  ',
 '좋은 책이 있으면 소개해 줘.  ',
 '최신 음악 트렌드 좀 알려줄래?  ',
 '요즘 날씨가 어떤지 궁금해.  ',
 '가벼운 운동 추천해 줄 수 있어?  ',
 '주말에 할 만한 액티비티가 뭐가 있을까?  ',
 '스트레스 푸는 방법 좀 알려줘.  ',
 '취미로 하기 좋은 것 추천해 줄 수 있어?  ',
 '간단한 요리 레시피 하나 알려줘.  ',
 '최근에 인기 있는 앱 있으면 추천해 줘.  ',
 '여행지로 가기 좋은 곳이 어디일까?  ',
 '친구한테 선물하기 좋은 아이템 뭐가 있을까?  ',
 '이 계절에 가볼 만한 카페 있어?  ',
 '요즘 유행하는 패션 트렌드가 뭐야?  ',
 '마음에 드는 음악 장르 추천해줘.  ',
 '편안하게 쉴 수 있는 장소가 있을까?  ',
 '영화관에서 볼 만한 좋은 영화가 뭐야?  ',
 '효과적인 공부 방법 좀 알려줄 수 있어?  ',
 '최근에 본 뉴스 중 재미있는 거 있어?  ',
 '간단하게 즐길 수 있는 게임 추천해 줘.  ',
 '긴장을 푸는 방법 뭐가 있을까?  ',
 '자기계발에 좋은 강좌 추천해 줄래?  ',
 '나만의 루틴 만들기에 대한 팁이 있을까?  ',
 '일상에서 행복을 느끼게 해주는 작은 것들이 뭐가 있을까?',
 '오늘 날씨 어때?  ',
 '요즘 인기 있는 영화 추천해줘.  ',
 '좋은 음악 플레이리스트 있으면 공유해줘.  ',
 '최근 트렌드 중에서 뭐가 제일 흥미로워 보여?  ',
 '주말에 가볼 만한 장소 추천해줄래?  ',
 '재미있는 팟캐스트 있으면 알려줘.  ',
 '요즘 읽기 좋은 책 있으면 추천해줘.  ',
 '차 한 잔 할 만한 카페 추천해줘.  ',
 '요즘 즐겨 보는 TV 프로그램은 뭐야?  ',
 '좋은 요리 레시피 있으면 알려줘.  ',
 '이런 날씨에 어울리는 음악은 뭐야?  ',
 '웹툰 중에서 추천할 만한 게 있어?  ',
 '최근에 인기 있는 게임 뭐가 있는지 알고 싶어.  ',
 '좋은 운동 방법이나 루틴이 있으면 알려줘.  ',
 '기분 전환에 좋은 활동 추천해줘.  ',
 '친구에게 선물할 만한 아이템 있을까?  ',
 '오늘 하루 마무리하는 좋은 방법이 있을까?  ',
 '제철 음식 중에 추천할 만한 것이 뭐야?  ',
 '기분이 좋을 때 듣기 좋은 음악은 뭐야?  ',
 '이색적인 여행지 추천해줄 수 있어?  ',
 '재미있는 일화나 유머가 있으면 들려줘.  ',
 '나만의 루틴을 만드는 데 도움이 될 만한 조언이 있을까?  ',
 '간단하게 할 수 있는 취미가 뭐가 있을까?  ',
 '여름에 어울리는 시원한 음료 추천해줘.  ',
 '마음이 편안해지는 방법이 있을까?',
 '오늘 날씨가 좋은데, 어떤 기분이 드는지 궁금해!  ',
 '요즘 가장 인기 있는 영화가 뭐인지 알고 있어?  ',
 '내가 한번 읽어볼 만한 좋은 책 추천해줄래?  ',
 '이 음악 어떤 느낌인지 한번 들어보고 싶어!  ',
 '여행 가기 좋은 곳이 어디인지 알려줄 수 있어?  ',
 '최근에 유행하는 패션 아이템이 뭐인지 궁금해!  ',
 '지금 듣고 있는 노래의 분위기가 어때?  ',
 '좋은 카페가 있으면 추천해줄 수 있어?  ',
 '요즘 핫한 레스토랑은 어디인지 알고 있어?  ',
 '요즘 인기 있는 게임 좀 알려줄래?  ',
 '내가 할 만한 취미 활동이 뭐가 있을까?  ',
 '최근에 재밌게 본 유튜브 채널이 뭐야?  ',
 '요즘 뜨고 있는 TV 프로그램이 있다면 뭐지?  ',
 '다양한 커피 종류 중 어떤 게 맛있을까?  ',
 '네가 추천하는 간식은 뭐야?  ',
 '최근에 본 전시회 중에서 추천할 만한 게 있을까?  ',
 '자주 가는 운동이 있으면 알려줄 수 있어?  ',
 '유명한 디저트 카페가 있다면 추천해줘!  ',
 '좋은 음악 추천해주면 듣고 싶어!  ',
 '주말에 할 만한 재미있는 활동이 있을까?  ',
 '요즘 보고 싶은 애니메이션이 있다면 뭐지?  ',
 '감명 깊었던 다큐멘터리 추천해줄 수 있어?  ',
 '혼자서 볼 수 있는 시리즈가 있다면 뭐가 좋을까?  ',
 '최근에 유행하는 패션 스타일이 있다면 알려줘!  ',
 '가볍게 할 수 있는 게임 중에서 추천할 만한 게 있을까?']

hello_list3 = []

for hello in hello_list2:
    hello = hello.strip()
    hello_list3.append(hello)


hello_list = hello_list3 + hello_list
hello_list


['오늘 기분이 어떤지 궁금해!',
 '요즘 인기 있는 음악 추천해 줄래?',
 '좋은 영화 있으면 알려줘!',
 '요즘 트렌드인 패션 아이템 뭐가 있을까?',
 '사무실에서 듣기 좋은 팟캐스트 추천해 줄 수 있어',
 '휴식 시간에 할 만한 간단한 게임 있으면 알려줘!',
 '최근에 화제가 된 뉴스는 뭐가 있을까?',
 '오늘 날씨 어때?',
 '내가 가볼 만한 여행지 추천해 줘!',
 '재밌는 책 읽고 싶은데, 추천할 만한 거 있어?',
 '요즘 인기 있는 TV 프로그램 있으면 알려줘!',
 '간단한 요리 레시피 알려줄래?',
 '주말에 놀러 갈 만한 장소 추천해 줘!',
 '요즘 핫한 카페나 맛집 정보 있으면 알려줘!',
 '기분 전환할 만한 좋은 방법이 있을까?',
 '새로운 취미를 시작해 보고 싶은데 뭐가 좋을까?',
 '사무실에 어울리는 음악 플레이리스트 추천해 줘!',
 '감동적인 영화 있으면 한 편 추천해 줘!',
 '인기 있는 앱이나 웹사이트 알려줄래?',
 '오늘의 긍정적인 메시지 하나 던져 줄래?',
 '가벼운 스트레칭 방법 추천해 줘!',
 '재밌는 유튜브 채널 알려줄 수 있어?',
 '감상할 만한 미술 전시회 있으면 추천해 줘!',
 '건강한 스낵 아이디어가 있다면 알려줘!',
 '요즘 사람들 사이에서 인기 있는 활동이 뭐지?',
 '오늘 날씨 어때?',
 '요즘 핫한 영화 추천해줄 수 있어?',
 '좋아하는 음악 있으면 알려줘!',
 '최근에 나온 책 중에 읽어볼 만한 거 있어?',
 '어떤 유튜브 채널 재미있어?',
 '오늘 기분 좋게 해줄 만한 명언 하나 알려줘!',
 '나에게 어울리는 디저트 추천해줘!',
 '주말에 할 만한 활동이 있을까?',
 '요즘 인기 있는 드라마 뭐야?',
 '나랑 잘 맞는 취미가 뭐라고 생각해?',
 '스트레스 해소하는 방법 몇 가지 알려줘!',
 '재미있는 팟캐스트 추천해줄래?',
 '다음에 가볼 만한 여행지 있으면 알려줘!',
 '내가 좋아할 만한 게임 뭐 있을까?',
 '나에게 맞는 카페 스타일은 

### 학습 데이터 생성함수

In [57]:
tool_prompt = """
사용자는 frontend(프론트엔드)팀에 속한 팀원입니다.
당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "frontend_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: \'코드노바 API 서버 설정\')"}}, "required": ["keyword"], "additionalProperties": false}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}'
</tool_call>
"""


system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 코드노바의 사내 문서 AI 챗봇입니다.

{tool_prompt}

그리고 다음 지침을 반드시 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사무적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자의 말투와 상관 없이, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
"""

SYSTEM_PROMPT = {
    "role": "system",
    "content": system_message
}

def parse_queries(resp_text: str, fallback: str) -> list[str]:
    try:
        obj = json.loads(resp_text)
    except json.JSONDecodeError:
        return [fallback]
    q = obj.get("queries", fallback)
    if isinstance(q, str):
        q = [q]
    elif isinstance(q, list):
        q = [s.strip() for s in q if isinstance(s, str) and s.strip()]
    else:
        q = [fallback]
    return q or [fallback]

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q

    query = True

    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})
        print(current_q)

        if current_q not in hello_list and query:
            response = query_chain.invoke({"history": messages})
            questions = parse_queries(response, fallback=current_q)

            tool_calls = []
            tool_responses = []
            for question in questions:
                tool_calls.append(f"<tool_call>{{\"name\": \"frontend_search\", \"arguments\": {{\"keyword\": \"{question}\"}}}}</tool_call>")

                # (2) 문서 검색 (vectorstore에서)
                k = randint(4,8)
                docs = vectorstore.similarity_search(question, k=k)
                if docs:
                    ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
                else:
                    ref_text = "검색 결과가 없습니다."
                ref_text = f"검색 결과:\n-----\n{ref_text}"
                tool_responses.append(f"<tool_response>{ref_text}</tool_response>")
            tool_call = "\n".join(tool_calls).strip()
            tool_response = "\n".join(tool_responses).strip()

            if tool_call:
                messages.append({"role": "assistant", "content": tool_call})
            if tool_response:
                messages.append({"role": "user", "content": tool_response})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            temperature=0.2
        )
        assistant_reply = (response.choices[0].message.content or "").strip()
        # [Fix-4] 비어 있으면 기본 문구로 대체(선택)
        if not assistant_reply:
            assistant_reply = "요청하신 내용과 관련된 정보를 찾지 못했습니다. 다른 키워드로 다시 말씀해 주시면 확인하겠습니다."
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):
            if choice([True, False]):
                complex1 = pick_complex2()
                followup = question_chain.invoke({
                "history": messages,"complex": complex1
            })
                current_q = followup
                query = True
            else:
                intent = pick_intent()
                complex2 = pick_complex()
                followup = simple_chain.invoke({"history": messages, "intent": intent, "complex": complex2})
                current_q = followup
                query = False
        else:
            break

    return {"messages": messages}

In [ ]:
import time
from tqdm import tqdm

train_dataset = []
for q in tqdm(initial_questions):
    time.sleep(1)  # 1초 대기
    sample = generate_one_chat_sample(q, max_turns=4)
    train_dataset.append(sample)

# 저장
with open(f"qwen3_company_train_dataset_{permission}3.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)


  0%|          | 0/336 [00:00<?, ?it/s]

내부 평가 피드백 문서에서 프론트엔드팀의 업무 수행 능력에 대한 성과는 어떤가요?
